In [1]:
import ipyleaflet 
import ipywidgets
import pandas as pd
import time
import os
import asyncio
import logging
from branca.colormap import linear
from random import uniform

markers = []
ANIMATION_STEP_DELAY = 0.2  # in seconds

m = ipyleaflet.Map(center = (50.0755,14.4), zoom = 10, layout=ipywidgets.Layout(height='720px'))

<H1>Heatmap vizualizace</H1>
<H3>Vizualizace koncentrace uživatelů, agregovaná na základě času</H3>
<br>
Vizualizace založeny na datech společnosti SmartGuide
<br>
Vytvořeno v rámci bakalářské práce pro Fakultu Informatiky a Managementu Univerzity Hradce Králové
<br>
<br>
Petr Veselý

In [2]:
data = pd.concat(map(pd.read_csv, ['times/times_0.csv', 'times/times_1.csv', 'times/times_2.csv']), ignore_index=True)
days = sorted(data.date.unique())
dev_languages=sorted(data.dev_language.unique())
dev_os=sorted(data.dev_system.unique())

In [3]:
slider = ipywidgets.SelectionRangeSlider(description='',
                                         options=days,
                                         readout=True,
                                         continuous_update=False, 
                                         layout=ipywidgets.Layout(width='720px')
)

In [4]:
os_select = ipywidgets.SelectMultiple(
    options= dev_os,
    description='Device OS',
    disabled=False
)

In [5]:
lang_select = ipywidgets.SelectMultiple(
    options= dev_languages,
    description='Language',
    rows=15,
    disabled=False
)

In [6]:
all_langs_button = ipywidgets.Button(
    description='All languages',
    disabled=False,
    tooltip='Select all languages',
    icon='fa-language',
    layout=ipywidgets.Layout(width='150px')
)

def all_langs(_):
    lang_select.value = dev_languages

all_langs_button.on_click(all_langs)

In [7]:
tourist_button = ipywidgets.ToggleButton(
    value=False,
    description='Tourists only',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Show only assumed tourists, only useful within Czechia',
    icon='fa-suitcase'
)

In [8]:
animate_button = ipywidgets.ToggleButton(
    description='',
    disabled=False,
    value=False,
    tooltip='Animate',
    icon='play', # pause,
    layout=ipywidgets.Layout(width='50px')
)

In [9]:
all_button = ipywidgets.Button(
    description='',
    disabled=False,
    tooltip='All Data!',
    icon='fa-globe',
    layout=ipywidgets.Layout(width='50px')
)

In [10]:
def call_update(change):
   update(slider.value, lang_select.value, os_select.value, tourist_button.value)

slider.observe(call_update, 'value')
os_select.observe(call_update, 'value')
lang_select.observe(call_update, 'value')
tourist_button.observe(call_update, 'value')

In [11]:
def animate(change):
    if(animate_button.value == True):
        animate_button.icon = 'pause'
    else:
        animate_button.icon = 'play'   

animate_button.observe(animate, 'value')

In [12]:
def all_data(change):
    print('Prepping full datagram, please wait.')
    update()
    
all_button.on_click(all_data)

In [13]:
def update(date=None, lang=None, os=None, tourist=None):
    if not date and not lang and not os:
        frame_data = data
        heat.max = data['count'].quantile(0.95)*len(days)
        
    else:
        frame_data = data.loc[data['date'] >= date[0]].loc[data['date'] <= date[1]]
        if os:
            frame_data = frame_data[frame_data['dev_system'].isin(os)]
        if not tourist:
            if lang:
                frame_data = frame_data[frame_data['dev_language'].isin(lang)]
        else:    
            frame_data = frame_data[frame_data.tourist.eq(True)]
        
        heat.max = data['count'].quantile(0.95) 
    markers = []
    longitudes = frame_data['gps_lon'].values.tolist()
    latitudes = frame_data['gps_lat'].values.tolist()
    nums = frame_data['count'].values.tolist()
    for lon, lat, num in zip(longitudes, latitudes, nums):
        markers.append([lat, lon, num])
    heat.locations = markers

In [14]:
def run_animation_step():
    i = 1
    j = 0
   
    while i < len(days) and i != days[i] != slider.value[1] :
        i += 1

    while j < i and j != days[j] != slider.value[0] :
        j += 1
        
    if i + 1 < len(days):
        slider.value = (days[j + 1], days[i+1] )
        return True
    
    animate_button.value = False
    return False

async def run_animation_loop():
    while True:
        await asyncio.sleep(ANIMATION_STEP_DELAY)
        if animate_button.value == True:
            run_animation_step()
    return x

In [15]:
heat = ipyleaflet.Heatmap(
    locations=[],
    radius=5,
    blur=2.5,
    min_opacity=0.2,
    gradient={
        0.4: 'blue',
        0.6: 'cyan',
        0.7: 'lime', 
        0.8: 'aquamarine',
        0.9: 'orange',
        1.0: 'red'}
)

m.add_layer(heat)
m.add_control(ipyleaflet.FullScreenControl())

In [16]:
widget_control1 = ipyleaflet.WidgetControl(widget=slider, position='topright', layout=ipywidgets.Layout(width='500px'))
m.add_control(widget_control1)
widget_control2 = ipyleaflet.WidgetControl(widget=animate_button, position='bottomleft')
m.add_control(widget_control2)
widget_control3 = ipyleaflet.WidgetControl(widget=all_button, position='bottomleft')
m.add_control(widget_control3)
widget_control4 = ipyleaflet.WidgetControl(widget=lang_select, position='bottomright')
m.add_control(widget_control4)
widget_control5 = ipyleaflet.WidgetControl(widget=os_select, position='bottomright')
m.add_control(widget_control5)
widget_control6 = ipyleaflet.WidgetControl(widget=tourist_button, position='topright')
m.add_control(widget_control6)
widget_control7 = ipyleaflet.WidgetControl(widget=all_langs_button, position='bottomright')
m.add_control(widget_control7)

In [17]:
display(m)

logging.getLogger('asyncio').setLevel(logging.WARNING)
asyncio.ensure_future(run_animation_loop())
slider.value = (days[0], days[3])
#run_animation_step()

Map(center=[50.0755, 14.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

Pro selekci více položek v ui modulech Device OS a Language lze použít klávesy Shift nebo CTRL.